In [ ]:
import pymysql.cursors
import pymysql
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
import datetime
# from datetime import datetime
import dash_bootstrap_components as dbc

In [ ]:
db_host = 'xxx'
db_user = 'xxx'
db_password = 'xxx'
db_database = 'xxx'



# Get all stock codes and stock company names

In [ ]:
def all_stock_codes_and_stock_company_names():
    connection = pymysql.connect(host=db_host,
                             user=db_user,
                             password=db_password,
                             db=db_database,
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
    dbcursor = connection.cursor()
    query="""
    SELECT sit.code, sit.company_name
    FROM stock_info_table sit;"""
    
    out_df = pd.read_sql(query, connection)
    connection.close()
    return out_df
    

In [ ]:
hsi_code_name_df = all_stock_codes_and_stock_company_names()

In [ ]:
def all_participant_info():
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 db=db_database,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    dbcursor = connection.cursor()
    query="""
    SELECT * FROM participant_info_table;"""
    
    out_df = pd.read_sql(query, connection)
    connection.close()
    return out_df
    

In [ ]:
participant_info_df = all_participant_info()

In [ ]:
def get_stock_value(company_name, start_date, end_date):
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 db=db_database,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    dbcursor = connection.cursor()
    query = """
        WITH sit AS(select sit.code
        from stock_info_table as sit
        where sit.company_name = '{}')
        SELECT svt.code, svt.date, svt.high, svt.low, svt.open, svt.close
        FROM sit 
        INNER JOIN stock_value_table svt ON sit.code=svt.code
        WHERE svt.date BETWEEN '{}' AND '{}';
    """.format(company_name, start_date, end_date)
    
    df = pd.read_sql(query,connection)
    connection.close()
    
    fig = go.Figure(
        data=[
                go.Candlestick(
                    x=df['date'],
                    open=df['open'], 
                    high=df['high'],
                    low=df['low'], 
                    close=df['close']
                )
             ],
        layout = go.Layout(
            autosize=True,
            title=company_name,
            yaxis_title='HK Dollar',
            xaxis_title='Date',
            template="plotly_white"
        ))
    
    return df,fig

In [ ]:
def get_stock_company_description(company_name):
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 db=db_database,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    dbcursor = connection.cursor()
    query = """
        SELECT sit.company_description
        FROM stock_info_table sit
        WHERE sit.company_name='{}'
    """.format(company_name)
    dbcursor.execute(query)
    connection.close()
    return dbcursor.fetchone()['company_description']

In [ ]:
def get_participant_amount(company_name, start_date, end_date):
    connection = pymysql.connect(host=db_host,
                                 user=db_user,
                                 password=db_password,
                                 db=db_database,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    dbcursor = connection.cursor()
    
    query = """
        WITH sit AS(select sit.code
        from stock_info_table as sit
        where sit.company_name = '{}')
        SELECT pit.participant_name, pat.date, pat.amount
        FROM sit
        INNER JOIN participant_amount_table pat on sit.code=pat.code
        INNER JOIN participant_info_table pit on pit.participant_id = pat.participant_id
        WHERE pat.date BETWEEN '{}' AND '{}';
    """.format(company_name, start_date, end_date)

    
    
    out_df = pd.read_sql(query,connection)
    connection.close()
    amount_df = out_df.groupby(['date','participant_name']).first().unstack(level=-1)
    
    fig = go.Figure(layout = go.Layout(
            autosize=True,
            title=company_name,
            yaxis_title='Amount',
            xaxis_title='Date',
            template="plotly_white"
        ))
    for c in amount_df.columns:
        fig.add_trace(
            go.Scattergl(
                x = amount_df.index,
                y = amount_df[c],
                mode = 'lines',
                marker = dict(
                    line = dict(width = 1)
                ),
                name=c[1],
                text=c[1]
            )
        )

    
    
    return fig, out_df['participant_name'].unique().tolist()

In [ ]:
def generate_participant_FormGroup(participant_name,participant_address):
    return dbc.FormGroup(
                [
                    html.H3(f'Stakeholder Name/Company: {participant_name}'),
                    html.Br(),
                    html.H3(f'Stakeholder Address: {participant_address}')
                ]
            ),
    

In [ ]:
stock_options = (hsi_code_name_df['company_name'].apply(lambda x:str(x))).tolist()
stock_options = [{'label':i, 'value':i}for i in stock_options]

# Navigation Bar

In [ ]:
navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Stock Candlestick Graph", href="/")),
        dbc.NavItem(dbc.NavLink("Stakeholder Graph", href="/stakeholder")),
    ],
    brand="Hang Seng Index Stock/Stakeholder Visualization Tools",
    brand_href="/",
    color="primary",
    dark=True,
)


# Control Panel

In [ ]:

controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                html.H3("Hang Seng Index Stock"),
                
                html.Div(
                    children=dcc.Dropdown(
                        id="stock_company",
                        options=stock_options,style={'fontColor':'black'}
                    ),
                    
                
                
                )
                
            ]
        ),
        dbc.FormGroup(
            [
                html.H5("Please select a start date and end date"),
                dcc.DatePickerRange(
                    id='my_date_picker',
                    min_date_allowed=datetime.date(2019, 1, 1),
                    max_date_allowed=datetime.date(2020, 9, 30),
                    start_date=datetime.date(2019,1,1),
                    end_date = datetime.date(2020,9,30)
                )
            ]
        ),
    ],
    body=True,
)


# Stock info

In [ ]:
stock_info = dbc.Card(
    [
        dbc.FormGroup(
            [
                html.H3("Description"),
                html.P(id='stock_company_description_id')
            ]
        ),
    ],
    body=True,
)

# Stock Statistic Info

In [ ]:
def generate_stock_statistic_info(stock_max, stock_min):
    stock_statistic = dbc.Card(
        [
            dbc.FormGroup(
                [
                    html.H1("Statistic"),
                    html.H2(f'Max:{stock_max}'),
                    html.Br(),
                    html.H2(f'Min:{stock_min}'),
                    html.Br(),
                ]
            ),
        ],
        body=True,
    )
    return stock_statistic

In [ ]:
def generate_participant_FormGroup(participant_name, participant_address):
    return dbc.Container(
                dbc.FormGroup(
                [
                    html.H5(f'Stakeholder Name/Company: {participant_name}'),
                    html.H5(f'Stakeholder Address: {participant_address}'),
                    html.Br(),
                ]
            ),
    
    
    )

In [ ]:
def generate_stakeholder_info(df):
    df = participant_info_df[participant_info_df['participant_name'].isin(df)]
    info = df[['participant_name','participant_address']].values.tolist()
    return dbc.Card([generate_participant_FormGroup(i[0],i[1]) for i in info], body=True)
    
    

# Frontend

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dcc.Location(id='url',refresh=False),
    navbar,
    dbc.Container([
        html.Br(),
        dbc.Row(
            [
                dbc.Col(controls,md=4),
                dbc.Col(dcc.Graph(id='graph_area',
                ),md=8)
            ]
        ),
    ],
    fluid=True),
    dbc.Container([
        html.Br(),
        dbc.Row(
            [
                dbc.Col(stock_info,md=4),
                dbc.Col(id='additional_info_area',
                        md=8),
            ],
        ),
    ],
    fluid=True)

])

# Backend

In [ ]:
@app.callback([Output('graph_area','figure'),
               Output('stock_company_description_id','children'),
               Output('additional_info_area','children')
               ],
             [Input('url','pathname'),
              Input('stock_company','value'),
              Input('my_date_picker','start_date'),
              Input('my_date_picker','end_date')])
def refresh(pathname, value, start_date, end_date):
    if(pathname=='/stakeholder'):
        fig, amount_df = get_participant_amount(value, start_date, end_date)
        
        
        return fig, get_stock_company_description(value), generate_stakeholder_info(amount_df)
        
        
    else:
        df, fig = get_stock_value(value, start_date, end_date)
        df = df.describe()
        stock_high = str(df['high']['max'])
        stock_low  = str(df['low']['min'])
        print("stock graph")
        return fig, get_stock_company_description(value), generate_stock_statistic_info(stock_high, stock_low)
    

In [ ]:
app.run_server(debug=False)